<a href="https://colab.research.google.com/github/dsivov/NEBULA2/blob/master/notebooks/gdrive_tempalte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nebula on Gradient Cluster.


*   Register on https://console.paperspace.com
*   Create new notebook with following parameters:

1. Workspace URL - path to Nebula Git 
2. Container - currently dsivov/nebula_gradient:v05 
3. Command - /run.sh

Notebook layout:
/notebooks - main Nebula code
/storage - permanent storage (models)

pre-installed packages are under conda nebula env (/opt/...)
if you want to add permanent package - use pip with --target parameter and point it to any directory under /storage. Any other path prefix will be deleted on next run

This part must be executed each time you connect to Gradient docker

In [ ]:
import os, sys

nebula_path = '/notebooks'
nebula_storage = '/storage'
isdir = os.path.isdir(nebula_path)

nebula_env = nebula_storage + "/env"
nebula_models = nebula_storage + "/models"
nebula_data = nebula_storage + "/data"

sys.path.insert(0, nebula_path)
sys.path.insert(0, nebula_env)
os.environ['TRANSFORMERS_CACHE'] = nebula_models


In [ ]:
#!pip install --target=$nebula_env python-arango==7.2.0
#!pip install --target=$nebula_env pymilvus==1.1.0

Example for MDMMT related packages

In [ ]:
# MDMMT packages, uncomment for first usage
#!pip install --target=$nebula_env  tf_slim 
#!pip install --target=$nebula_env ftfy
#!pip install --target=$nebula_env transformers

In [3]:
from nebula_api.nebula_enrichment_api import NRE_API

In [ ]:

from nebula_api.mdmmt_api.mdmmt_api import MDMMT_API
mdmmt = MDMMT_API()


Tests:


1.   DataBase connector
2.   MDMMT 
3.   Milvus API



In [ ]:
class DatabaseConnector():
  def __init__(self):
      self.arango_host = 'http://ec2-18-158-123-0.eu-central-1.compute.amazonaws.com:8529'

  def connect_db(self, dbname):
      client = ArangoClient(hosts=self.arango_host)
      db = client.db(dbname, username='nebula', password='nebula')
      return (db)
    

In [ ]:
gdb = DatabaseConnector()
db = gdb.connect_db("nebula_development")


In [ ]:
def get_movie_url(movie_id, db):
  #nebula_movies = []
  url = ""
  query = 'FOR doc IN Movies FILTER doc._id == "{}" RETURN doc.url_path'.format(movie_id)
  cursor = db.aql.execute(query)
  for data in cursor:
      #print(data)
      url = data
  #print(url)
  return(url)

In [ ]:
get_movie_url('Movies/114208149', db)

'static/development/1035_The_Adjustment_Bureau_00_01_40_825-00_01_46_814.mp4'

In [5]:
vector = mdmmt.encode_text("test encoder")
print(vector)

tensor([ 0.0050, -0.0057,  0.0204,  ...,  0.0024,  0.0030, -0.0023],
       device='cuda:0')


Test Nebula main API

In [ ]:
nre = NRE_API()
nre.get_all_movies()

Test Milvus API

In [ ]:
from config_nebula.config import NEBULA_CONF
ncfg = NEBULA_CONF()
HOST, _PORT = ncfg.get_milvus_server()
print(HOST, " ", _PORT)
from nebula_api.milvus_api import MilvusAPI
milvus_events = MilvusAPI(
            'milvus', 'vcomet_mdmmt_embedded_event', 'nebula_visualcomet', 1536)
vector = vector.tolist()
milvus_events.search_vector(40, vector)